In [4]:
import pandas as pd
import ast
import glob

In [25]:
df = pd.read_csv('10000recipe_for_db.csv')
type_counts = df['type_key'].value_counts()
print(type_counts)

type_key
반찬         37217
밥/죽/떡      13628
국/찌개        8014
면/만두        6415
빵/과자        6098
디저트         4543
양식          3612
샐러드         2432
퓨전          1355
양념/잼/소스      909
스프           779
차/음료/술       478
Name: count, dtype: int64


In [36]:
df = pd.read_csv('10000recipe_for_db.csv')
banchan_df = df[(df['type_key'] == '반찬') & (df.index >= 8000)]
type_counts = banchan_df['method_key'].value_counts()
print(type_counts)

method_key
볶음     6864
굽기     4431
부침     4302
찜      3216
무침     2720
조림     2531
튀김     2022
끓이기    2017
삶기      697
절임      252
회       107
비빔       58
Name: count, dtype: int64


In [63]:
df = pd.read_csv('recipe_analysis_checkpoint_20241107_091855.csv')
def safe_eval(x):
    try:
        return ast.literal_eval(x) if pd.notna(x) else []
    except:
        return []

df['recipe_type'] = df['recipe_type'].apply(safe_eval)

# 빈 리스트인 행 제거
df = df[df['recipe_type'].apply(len) > 0]
# banchan_df = df[df['type_key'] == '반찬']
type_counts = df['type_key'].value_counts()
print(type_counts)

type_key
반찬    7900
Name: count, dtype: int64


In [67]:
df = pd.read_csv('jori_recipe_for_db.csv')
type_counts = df['type_key'].value_counts()
print(type_counts)
df.info()

type_key
반찬         478
밥/죽/떡      129
국/찌개        97
면/만두        87
양식          76
빵/과자        76
샐러드         68
디저트         43
스프          26
차/음료/술      23
퓨전           7
양념/잼/소스      1
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          1111 non-null   object 
 1   main_img       1111 non-null   object 
 2   type_key       1111 non-null   object 
 3   method_key     1111 non-null   object 
 4   servings       1111 non-null   int64  
 5   cooking_time   0 non-null      float64
 6   difficulty     0 non-null      float64
 7   ingredient     1111 non-null   object 
 8   cooking_order  1111 non-null   object 
 9   cooking_img    1111 non-null   object 
 10  hashtag        1111 non-null   object 
 11  tips           1111 non-null   object 
 12  recipe_type    1111 non-null   object 
dtypes: float64(2)

In [28]:
df = pd.read_csv('recipe_analysis_checkpoint_20241107_091855.csv')
def safe_eval(x):
    try:
        return ast.literal_eval(x) if pd.notna(x) else []
    except:
        return []

df['recipe_type'] = df['recipe_type'].apply(safe_eval)

# 빈 리스트인 행 제거
df = df[df['recipe_type'].apply(len) > 0]
df.to_csv('recipe_analysis_checkpoint_20241107_091855.csv', index=False)

In [2]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
df = pd.read_csv('10000recipe_for_db.csv')

banchan_df = df[(df.index >= 8000) & (df['type_key'] == '샐러드')]

# 필터링된 데이터에서 method_key 비율 계산
method_counts = banchan_df['method_key'].value_counts()
total = method_counts.sum()

# 비율 계산 및 소수점 처리
raw_ratios = {method_type: (count/total) * 800 for method_type, count in method_counts.items()}
ratios = {k: int(v) for k, v in raw_ratios.items()}

# 합계가 800이 되도록 조정
current_sum = sum(ratios.values())
difference = 800 - current_sum

if difference > 0:
    # 소수점 부분이 가장 큰 순서대로 1씩 더해줌
    decimal_parts = {k: v - int(v) for k, v in raw_ratios.items()}
    sorted_types = sorted(decimal_parts.keys(), key=lambda x: decimal_parts[x], reverse=True)
    
    for i in range(difference):
        ratios[sorted_types[i]] += 1

# 각 타입별로 샘플링하여 결과 저장
sampled_dfs = []
for method_type, sample_size in ratios.items():
    type_df = banchan_df[banchan_df['method_key'] == method_type]
    if len(type_df) >= sample_size:
        sampled = type_df.sample(n=sample_size, random_state=42)
    else:
        sampled = type_df  # 만약 해당 타입의 데이터가 필요한 샘플 수보다 적다면 전부 사용
    sampled_dfs.append(sampled)

# 모든 샘플을 하나의 데이터프레임으로 합치기
final_sample = pd.concat(sampled_dfs)

# 결과 확인
print("최종 샘플 크기:", len(final_sample))
print("\n타입별 분포:")
print(final_sample['method_key'].value_counts())

# 결과 저장 (필요한 경우)
final_sample.to_csv('banchan_samples7.csv', index=False)

최종 샘플 크기: 800

타입별 분포:
method_key
무침     402
굽기     101
삶기      86
비빔      71
볶음      37
튀김      24
찜       22
절임      22
회       13
끓이기     12
부침       8
조림       2
Name: count, dtype: int64


In [5]:
# CSV 파일들의 경로 패턴 지정
file_pattern = 'banchan_sample*.csv'  # banchan_sample로 시작하는 모든 csv 파일

# 모든 파일 목록 가져오기
all_files = glob.glob(file_pattern)

# 데이터프레임을 저장할 리스트 생성
df_list = []

# 각 파일을 읽어서 리스트에 추가
for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

# 모든 데이터프레임을 하나로 통합
combined_df = pd.concat(df_list, ignore_index=True)

# 통합된 파일 저장
combined_df.to_csv('combined_banchan_samples.csv', index=False)

# 결과 확인
print("통합된 파일의 총 행 수:", len(combined_df))
print("\n각 파일별 행 수:")
for file in all_files:
    df = pd.read_csv(file)
    print(f"{file}: {len(df)} 행")

통합된 파일의 총 행 수: 9257

각 파일별 행 수:
banchan_samples11.csv: 478 행
banchan_samples10.csv: 779 행
banchan_samples7.csv: 800 행
banchan_samples6.csv: 800 행
banchan_samples4.csv: 800 행
banchan_samples5.csv: 800 행
banchan_samples1.csv: 800 행
banchan_samples2.csv: 800 행
banchan_samples3.csv: 800 행
banchan_samples8.csv: 800 행
banchan_samples9.csv: 800 행
banchan_samples.csv: 800 행


In [37]:
df = pd.read_csv("final_recipes.csv", encoding='utf-8', low_memory=False)
df['recipe_type'] = df['recipe_type'].apply(ast.literal_eval)
type_counts = pd.Series([item for sublist in df['recipe_type'] for item in sublist]).value_counts(dropna=False)
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
print(type_counts)

고단백       9721
한식        9513
저칼로리      7411
반찬        4557
매운맛1단계    4395
해산물       4196
볶음        2761
채식        2528
간편식       2032
조림        1770
새콤달콤한맛    1590
양식        1502
주식        1486
건강식       1484
국         1438
고섬유       1293
무침        1264
디저트       1207
구이        1201
비타민       1186
전         1158
저염식       1134
퓨전        1078
담백한맛       967
찜          962
튀김         914
샐러드        858
부침         844
스프         725
전통식        546
음료         458
매운맛2단계     374
소화촉진       205
저당식        195
체중조절       179
일식         151
장건강        112
특별식        105
가벼운        104
중식         103
삶기          96
죽           83
글루텐프리       70
매운맛3단계      63
매운맛4단계      31
저탄수화물       27
무기질         22
생식          16
매운맛5단계       4
Name: count, dtype: int64


In [30]:
mapping = {
    '다이어트에 좋은': '체중조절',
    '다양한 식이섬유': '고섬유',
    '다이어트샐러드': '체중조절',
    '다이어트 요리': '체중조절',
    '비건요리': '채식',
    '상큼달콤한맛': '새콤달콤한맛'
}

df['recipe_type'] = df['recipe_type'].apply(lambda x: [mapping.get(i, i) for i in x])

In [31]:
# 제외하고 싶은 타입들을 리스트로 정의
remove_categories = [
    '채소', '고구마', '고칼로리', '찬', '피클', '다이어트', '다이어트식', '달콤한맛', '고추장', '계란요리', '다이어트음식', '다이어트식단', '겨울호박으로 별미', '고탄수화물', '유기농',
    '계란', '계란말이', '한국식', '달콤짭짤한맛', '단짠단짠', '계란 요리', '다이어트 음식', '비건', '고급 요리', '비건식', '아이반찬', '고소한맛', '부드러운', '상큼한맛', '염증치료',
    '찌기', '고추가루', '다이어트요리', '고춧가루', '단짠단짠한맛', '구수한맛', '봄나물', '해독주스', '과일', '단짠한맛', '보양식', '브런치', '비빔', '냉채', '향신료', '발효식품',
    '여름식', '고구이', '디톡스', '고소짭쟐', '고기구이', '준비식', '달달한맛', '비건파스타', '즉석식', '아삭아삭한식', '프루트잼', '발효식', '영양수프', '무카페인', '비건라떼',
    '디톡스주스', '허브차', '민트차', '비건 스프', '다양한 영양소', '다이어트 스프', '다이어트스프', '유치원 초등 아이 간식', '푸드'
]

df['recipe_type'] = df['recipe_type'].apply(lambda x: [item for item in x if item not in remove_categories])
df['recipe_type'] = df['recipe_type'].apply(lambda x: list(dict.fromkeys(x)))

In [32]:
df.to_csv('samples.csv', index=False)

In [39]:
import pandas as pd

df1 = pd.read_csv('jori_recipe_for_db.csv')
df2 = pd.read_csv('final_recipes.csv')

# 데이터프레임 합치기
combined_df = pd.concat([df1, df2], ignore_index=True)

# 결과를 CSV 파일로 저장
combined_df.to_csv('final_recipes.csv', index=False, encoding='utf-8-sig')

print(f'처리 완료!\n- 첫 번째 파일 행 수: {len(df1)}\n- 두 번째 파일 행 수: {len(df2)}\n- 병합 후 전체 행 수: {len(combined_df)}')

처리 완료!
- 첫 번째 파일 행 수: 1111
- 두 번째 파일 행 수: 17157
- 병합 후 전체 행 수: 18268
